In [407]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

# Получение данных

- пометка на удаление = нет как для ном так и для родителя
- стоп продажа = нет как для ном так и для родителя
- тип номенклатуры = товар
- есть цена
- цена 0 или отсутсвует, то ее как бы нет

In [2]:
path = '~\\Полные аналоги номенклатуры на основе правил\\'
original_items = pd.read_excel(path + 'Реквизиты номенклатуры.xlsx', dtype = {'Код2':'object'})
original_prices = pd.read_excel(path + 'Номенклатура с ценой.xlsx', dtype = {'Код2':'object'}, header = 4).iloc[1:]
original_prices.rename(columns = {'Розничные':'price', 'Unnamed: 7':'currency'}, inplace = True)
original_items.info()
original_pri
fo()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286263 entries, 0 to 286262
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Код2                286263 non-null  object
 1   Артикул             282937 non-null  object
 2   Номенклатура        286263 non-null  object
 3   Пометка удаления    286263 non-null  object
 4   Тип номенклатуры    282937 non-null  object
 5   Пометка удаления.1  286263 non-null  object
 6   Родитель            286198 non-null  object
 7   Владелец            286263 non-null  object
 8   Свойство            286263 non-null  object
 9   Значение            274191 non-null  object
 10  Единица измерения   128201 non-null  object
dtypes: object(11)
memory usage: 24.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179060 entries, 1 to 179060
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Код2   

In [3]:
analog = pd.read_excel(path + 'Аналоги 1С.xlsx')
analog.rename(columns = {'Код2.1':'Код2 аналог'}, inplace = True)
analog.info()
analog.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273 entries, 0 to 5272
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Код2          5273 non-null   int64 
 1   Номенклатура  5273 non-null   object
 2   Код2 аналог   5273 non-null   int64 
 3   Аналог        5273 non-null   object
dtypes: int64(2), object(2)
memory usage: 164.9+ KB


,Код2,Номенклатура,Код2 аналог,Аналог
0,127066,Тележка инструментальная 7 полочная GARWIN Sta...,127068,Тележка инструментальная 7 полочная GARWIN Sta...


In [4]:
original_prices['currency'].unique()

array(['УЕ_Роз/USD', nan, 'руб.', 'У.Е./KN'], dtype=object)

In [5]:
current_exchange_rate = {} # словарь с курсами валют
current_exchange_rate['УЕ_Роз/USD'] = 122
current_exchange_rate['У.Е./KN'] = 115
current_exchange_rate['руб.'] = 1
current_exchange_rate

{'УЕ_Роз/USD': 122, 'У.Е./KN': 115, 'руб.': 1}

In [6]:
# перерасчет цен в рубли

prices_clean = original_prices[original_prices['currency'].isna() == False]
prices_clean['Цена'] = prices_clean.apply(lambda x: current_exchange_rate[x.loc['currency']] * x.loc['price'], axis = 1)
prices_clean.head(1)

C:\Users\MARYAN~1.D\AppData\Local\Temp/ipykernel_12536/582063299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_clean['Цена'] = prices_clean.apply(lambda x: current_exchange_rate[x.loc['currency']] * x.loc['price'], axis = 1)


,Код2,Артикул,Unnamed: 2,Unnamed: 3,Unnamed: 4,Номенклатура,price,currency,Unnamed: 8,Цена
1,1,1797513,NaN,NaN,NaN,10 мм торкс 75 mmL M13 (Фиат),1.9,УЕ_Роз/USD,28.02.2022,231.8


In [7]:
# объединение свойств с ценами

items = original_items.merge(prices_clean[['Код2', 'Цена']], how = 'left', on = 'Код2')
items.head()

,Код2,Артикул,Номенклатура,Пометка удаления,Тип номенклатуры,Пометка удаления.1,Родитель,Владелец,Свойство,Значение,Единица измерения,Цена
0,116597,"121003-М20*1,5","Метчик м/р М 20х1,5 ...",Нет,Товар,Нет,Метчики,"Метчик м/р М 20х1,5 ...",Тип резьбы,M - Метрическая,NaN,997.96
1,116597,"121003-М20*1,5","Метчик м/р М 20х1,5 ...",Нет,Товар,Нет,Метчики,"Метчик м/р М 20х1,5 ...",Стандарт,ГОСТ 3266-81,NaN,997.96
2,116597,"121003-М20*1,5","Метчик м/р М 20х1,5 ...",Нет,Товар,Нет,Метчики,"Метчик м/р М 20х1,5 ...",Шаг резьбы,"1,5 мм",NaN,997.96
3,116597,"121003-М20*1,5","Метчик м/р М 20х1,5 ...",Нет,Товар,Нет,Метчики,"Метчик м/р М 20х1,5 ...",Направление резьбы,Правая,NaN,997.96
4,116597,"121003-М20*1,5","Метчик м/р М 20х1,5 ...",Нет,Товар,Нет,Метчики,"Метчик м/р М 20х1,5 ...",Номинальный диаметр резьбы,M20,NaN,997.96


# Первичный анализ

## Повторения свойств товара

In [175]:
items.groupby(by = ['Код2', 'Артикул ', 'Свойство']).agg({'Значение':'count'}).reset_index().query('Значение > 1')#.to_excel(path + 'Дублируются свойства.xlsx')

,Код2,Артикул,Свойство,Значение
2278,100820,БГД-25,Максимальное рабочее давление газа,2
2288,100822,БКД-25,Максимальное рабочее давление газа,2
4424,102026,363-05-6-П,Диапазон измерений,2
4435,102027,363-07-6-П,Диапазон измерений,2
6976,103562,БКО 50-5,Максимальное рабочее давление газа,2
...,...,...,...,...
280221,98830,313-30-4-П,Диапазон измерений,2
280232,98831,313-40-4-П,Диапазон измерений,2
282240,99754,311-30-6-П,Диапазон измерений,2
282251,99755,311-12-0-П,Диапазон измерений,2


In [176]:
items.query('Код2 == "100820"')

,Код2,Артикул,Номенклатура,Пометка удаления,Тип номенклатуры,Пометка удаления.1,Родитель,Владелец,Свойство,Значение,Единица измерения,Цена
110991,100820,БГД-25,Редуктор газовый двухступенчатый БГД-25 (для н...,Нет,Товар,Нет,Редукторы баллонные двухступенчатые,Редуктор газовый двухступенчатый БГД-25 (для н...,Максимальное рабочее давление газа,"0,8 МПа",NaN,NaN
110992,100820,БГД-25,Редуктор газовый двухступенчатый БГД-25 (для н...,Нет,Товар,Нет,Редукторы баллонные двухступенчатые,Редуктор газовый двухступенчатый БГД-25 (для н...,Входное соединение,W21.8-1/14” или G3/4,NaN,NaN
110993,100820,БГД-25,Редуктор газовый двухступенчатый БГД-25 (для н...,Нет,Товар,Нет,Редукторы баллонные двухступенчатые,Редуктор газовый двухступенчатый БГД-25 (для н...,Выходное соединение,"М16х1,5",NaN,NaN
190700,100820,БГД-25,Редуктор газовый двухступенчатый БГД-25 (для н...,Нет,Товар,Нет,Редукторы баллонные двухступенчатые,Редуктор газовый двухступенчатый БГД-25 (для н...,Максимальная пропускная способность,25,м³/ч,NaN
251057,100820,БГД-25,Редуктор газовый двухступенчатый БГД-25 (для н...,Нет,Товар,Нет,Редукторы баллонные двухступенчатые,Редуктор газовый двухступенчатый БГД-25 (для н...,Габаритные размеры,183х172х101,мм,NaN
275807,100820,БГД-25,Редуктор газовый двухступенчатый БГД-25 (для н...,Нет,Товар,Нет,Редукторы баллонные двухступенчатые,Редуктор газовый двухступенчатый БГД-25 (для н...,Максимальное рабочее давление газа,20,МПа,NaN


In [90]:
code2_with_anomaly = items.groupby(by = ['Код2', 'Свойство']).agg({'Значение':'count'}).reset_index() \
                                                             .query('Значение > 1')['Код2'].unique()

#  Решение

##  Подготовка данных

In [14]:
# ДФ с артикулами и свойствами для расчета

act_items = items[(items['Пометка удаления'] == 'Нет') &
                  (items['Пометка удаления.1'] == 'Нет') &
                  (items['Тип номенклатуры'] == 'Товар') &
                  (items['Родитель'].isna() == False) &
                  (items['Цена'].isna() == False) &
                  (items['Значение'].isna() == False)] \
                .drop(columns = ['Пометка удаления', 'Тип номенклатуры', 'Пометка удаления.1'])
act_items.info()
act_items.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263254 entries, 0 to 286262
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Код2               263254 non-null  object 
 1   Артикул            263254 non-null  object 
 2   Номенклатура       263254 non-null  object 
 3   Родитель           263254 non-null  object 
 4   Владелец           263254 non-null  object 
 5   Свойство           263254 non-null  object 
 6   Значение           263254 non-null  object 
 7   Единица измерения  115698 non-null  object 
 8   Цена               263254 non-null  float64
dtypes: float64(1), object(8)
memory usage: 20.1+ MB


,Код2,Артикул,Номенклатура,Родитель,Владелец,Свойство,Значение,Единица измерения,Цена
0,116597,"121003-М20*1,5","Метчик м/р М 20х1,5 ...",Метчики,"Метчик м/р М 20х1,5 ...",Тип резьбы,M - Метрическая,NaN,997.96


In [15]:
# Создание множества свойств для каждого аортикула

chars = {} # словарь с множеством свойств каждого артикула
for c in act_items['Код2'].unique():
    props = set(act_items[act_items['Код2'] == c]['Свойство'].tolist())
    chars[c] = props

## Словарь со свойствами групп

In [18]:
rules = {} # словарь с множеством свойств для категории
rules['Трещотки'] = {'Размер привода', 'Количество зубьев', 'L - Длина общая'}
rules['Наборы ключей'] = {'Размер ключей в наборе', 'Количество предметов в наборе', 'Тип инструмента'}
rules['Наборы торцевых головок'] = {'Размер приводного квадрата', 'Размер', 'Тип головки', 'Рабочий профиль',
                                    'Количество предметов в наборе'}
rules['Наборы шестигранных ключей'] = {'Количество предметов в наборе', 'Размер', 'Рабочий профиль'}
rules['Наборы бит'] = {'Количество предметов в наборе', 'Рабочий профиль', 'Размер приводного квадрата',
                      'Размер приводного шестигранника'}
rules['Обжим'] = {'Тип клемм, наконечников, разъемов', 'Диапазон сечений'}
rules['Воротки '] = {'Тип воротка', 'Размер привода', 'L - Длина общая'}
rules['Наборы отверток'] = {'Количество предметов в наборе', 'Тип отвертки', 'Тип наконечника'}
rules['Кувалды'] = {'Вес бойка', 'Длина рукоятки'}
rules['Ручные тиски'] = {'Тип', 'Назначение', 'Диапазон захвата', 'L - Длина общая'}
rules['Бокорезы и кусачки'] = {'Изолированные рукоятки VDE (до 1000 В)', 'Тип инструмента',
                               'L - Длина общая'}
rules['Щипцы для стопорных колец'] = {'Тип стопорных колец', 'Диаметр стопорных колец', 'L - Длина общая'}
rules['Отвертки'] = {'Тип отвертки', 'Тип отвертки', 'Размер наконечника', 'Длина стержня'}
rules['Переставные клещи'] = { 'L - Длина общая', 'Размер'}
rules['Пассатижи'] = {'Диэлектрические рукоятки', 'L - Длина общая'}
rules['Серповидные ключи'] = {'Размер', 'Длина рукоятки'}
rules['Трещоточные ключи'] = {'Количество зубьев', 'Тип ключа', 'Размер', 'L - Длина общая'}
rules['Ключи трубные'] = {'Размер', 'L - Длина общая'}


rules_abs = {} # словарь с множеством свойств полного соответствия
rules_abs['Наборы ключей'] = {'Размер ключей в наборе', 'Количество предметов в наборе', 'Тип инструмента'}
rules_abs['Наборы торцевых головок'] = {'Размер приводного квадрата', 'Размер', 'Тип головки', 'Рабочий профиль',
                                        'Количество предметов в наборе'}
rules_abs['Наборы шестигранных ключей'] = {'Количество предметов в наборе', 'Размер', 'Рабочий профиль'}
rules_abs['Наборы бит'] = {'Количество предметов в наборе', 'Рабочий профиль', 'Размер приводного квадрата',
                           'Размер приводного шестигранника'}
rules_abs['Обжим'] = {'Тип клемм, наконечников, разъемов', 'Диапазон сечений'}
rules_abs['Воротки '] = {'Тип воротка', 'Размер привода'}
rules_abs['Наборы отверток'] = {'Количество предметов в наборе', 'Тип отвертки', 'Тип наконечника'}
rules_abs['Ручные тиски'] = {'Тип', 'Назначение'}
rules_abs['Бокорезы и кусачки'] = {'Изолированные рукоятки VDE (до 1000 В)', 'Тип инструмента'}
rules_abs['Щипцы для стопорных колец'] = {'Тип стопорных колец', 'Диаметр стопорных колец'}
rules_abs['Отвертки'] = {'Тип отвертки', 'Тип отвертки', 'Размер наконечника'}
rules_abs['Пассатижи'] = {'Диэлектрические рукоятки'}
rules_abs['Трещоточные ключи'] = {'Количество зубьев', 'Тип ключа', 'Размер'}


rules_about = {} # словарь с множеством свойств неполного соответсвия
rules_about['Трещотки'] = {'Количество зубьев', 'L - Длина общая'}
rules_about['Воротки '] = {'L - Длина общая'}
rules_about['Кувалды'] = {'Вес бойка', 'Длина рукоятки'}
rules_about['Ручные тиски'] = {'L - Длина общая'}
rules_about['Бокорезы и кусачки'] = {'L - Длина общая'}
rules_about['Щипцы для стопорных колец'] = {'L - Длина общая'}
rules_about['Отвертки'] = {'Длина стержня'}
rules_about['Переставные клещи'] = { 'L - Длина общая', 'Размер'}
rules_about['Пассатижи'] = {'L - Длина общая'}
rules_about['Серповидные ключи'] = {'Размер', 'Длина рукоятки'}
rules_about['Трещоточные ключи'] = {'L - Длина общая'}
rules_about['Ключи трубные'] = {'Размер', 'L - Длина общая'}


rules_deviation = {} # словарь с допустимым отклонением "неполного соответствия"
rules_deviation['Трещотки'] = {'Количество зубьев':0.35, 'L - Длина общая':0.05}
rules_deviation['Воротки '] = {'L - Длина общая':0.1}
rules_deviation['Кувалды'] = {'Вес бойка':0.1, 'Длина рукоятки':0.1}
rules_deviation['Ручные тиски'] = {'L - Длина общая':0.1}
rules_deviation['Бокорезы и кусачки'] = {'L - Длина общая':0.15}
rules_deviation['Щипцы для стопорных колец'] = {'Диаметр стопорных колец':0.1, 'L - Длина общая':0.15}
rules_deviation['Отвертки'] = {'Длина стержня':0.15}
rules_deviation['Переставные клещи'] = { 'L - Длина общая':0.15, 'Размер':0.1}
rules_deviation['Пассатижи'] = {'L - Длина общая':0.1}
rules_deviation['Серповидные ключи'] = {'Размер':0.1, 'Длина рукоятки':0.15}
rules_deviation['Трещоточные ключи'] = {'L - Длина общая':0.15}
rules_deviation['Ключи трубные'] = {'Размер':0.15, 'L - Длина общая':0.15}


rules_contains = {} # словарь который содержит слова, которые должны встречаться в названии номенклатуры
rules_contains['Кувалды'] = ['обратной']
rules_contains['Ручные тиски'] = ['образные', 'закругленными']
rules_contains['Щипцы для стопорных колец'] = ['загнутые']
rules_contains['Пассатижи'] = ['плоскогубцы', 'силовые']
rules_contains['Серповидные ключи'] = ['1/2', '3/4', '1/4']
rules_contains['Трещоточные ключи'] = ['шарниром']
rules_contains['Ключи трубные'] = ['американск', 'ремешковый', 'цепной', 'усиленный', 'колонковых',
                                  'буровых', 'цельнокованный', 'самозахватом']

## Функции для подбора

In [378]:
# функция создания ДФ номенклатуры, с которой будем работать.
# либо она работает только со свойствами, переданными ей из группы
# либо в качестве фильра добавляется ключевое слово которое используется для фильтрации данных по родителю

def act_art(group, parent = None):
    art_list = []
    
    # создаем список артикулов, которые удовлетворяют заданному набору свойств
    for c in chars.keys(): 
        if rules[group].issubset(chars[c]):
            art_list.append(c)
                        
    
    # формируем ДФ артикулов, которые мы будем рассматривать для нахождения аналогов
    if parent == None:
        df_art_list = act_items[(act_items['Код2'].isin(art_list))]
    else:
        df_art_list = act_items[(act_items['Код2'].isin(art_list)) &
                                (act_items['Родитель'].str.contains(parent, case = False))]
    
    group_amount = len(act_items[act_items['Родитель'].str.contains(group, case = False)]['Код2'].unique())
    sample_amount = len(df_art_list['Код2'].unique())
        
    print(f'Количество артикулов, которые пренадлежат родителю "{group}":',
          group_amount)
    print('Количество артикулов, которые попали в выборку:',
         sample_amount)
    print()
    
    if sample_amount / group_amount > 3:
        print('Согласно установленным правилам в отбор попадает слишком большое артикулов,',
             f'которые не пренадлежат родителю "{group}". Кажется, что в этом случае правила слишком общие и их стоит уточнить.',
             f'Установим ограничение на выборку только из соответствующего родителя "{group}"', sep = '\n')
        print()
        df_art_list = act_art(group, parent = group)
    
    return df_art_list

In [379]:
# функция нахождения пересечений множеств артикулов по свойствам

def intersection(**kwargs):
    art_subs_inter = {}
    for key, dict_dict in kwargs.items():
        list_set = list(dict_dict.values())
        art_subs_inter[key] = list_set[0].intersection(*list_set[1:])
    return art_subs_inter

In [380]:
# Запишем для каждого артикула мно-во артикулов, цены которых отклоняются в допустимом пределе

# Функция, которая подбирает множество аналогов для артикула по цене
# Принимает ДФ содержащий интересующие артикулы с ценой и dev - допустимое отклонение по цене, в рамках котороги мы будем
# подбирать аналог. Базово dev = 0,35

def subs_price(df, dev = 0.35):
    art_subs_price = {}
    df_art_list_price = df.groupby(by = 'Код2').agg({'Цена':'mean'})
    
    # ищем аналоги по цене для каждого артикула
    # ищем цену рассматриваемого артикула
    for art in df['Код2'].unique():
        price = df_art_list_price.loc[art][0]
        price_low = price * (1 - dev)
        price_hight = price * (1 + dev)
        
        # подбираем аналоги по цене
        price_subs = 0
        subs = set()      
        for art_subs in df['Код2'].unique():
            if art == art_subs:
                continue
            else:
                price_subs = df_art_list_price.loc[art_subs][0]
    
            if price_subs >= price_low and price_subs <= price_hight:
                subs.add(art_subs)
                
        art_subs_price[art] = subs
        
    return art_subs_price

In [381]:
# Функция подбора аналогов по категории признака "неполное соответсвие"

def subs_about(df, group):
    # проверяем, есть ли правила "неполного соответсвия" для этой группы
    try: 
        rules_about[group] # если есть группа в словаре правил "неполного соответсвия"
        art_subs_props_about = {}
        
        for art in df['Код2'].unique():
            prop_dict = {}
            
            for prop in rules_about[group]:
                value =  df[(df['Код2'] == art) & (df['Свойство'] == prop)]['Значение'].tolist()[0]
                
                 # проверяем тип данных значения, и меняем его на нужный
                if type(value) == type(str()):
                    try:
                        value = float(re.search(r'\d+', value)[0])
                    except:
                        value = 0
                        
                value_low = value * (1 - rules_deviation[group][prop])
                value_hight = value * (1 + rules_deviation[group][prop])
                
                subs = set() # множество, в которое мы будем записывать артикулы заменителей неполного соответсвия
                
                # получаем значения того же свойства для других артикулов
                value_sub = 0
                for art_sub in df['Код2'].unique():
                    if art == art_subs:
                        continue

                    else:
                        value_sub = df[(df['Код2'] == art_sub) &
                                        (df['Свойство'] == prop)]['Значение'].tolist()[0]
                        
                    # проверяем тип данных значения свойства этого аналога
                    if type(value_sub) == type(str()):
                        try:
                            value_sub = float(re.search(r'\d+', value_sub)[0])
                        except:
                            value_sub = 0
                    if value_sub >= value_low and value_sub <= value_hight:
                        subs.add(art_sub)
                
                # записываем множество аналогов текущего артикула по текущему свойству в словарь свойств
                prop_dict[prop] = subs
            
            # записываем словарь свойств с аналогами в словарь к артикулу
            art_subs_props_about[art] = prop_dict
            
        # используем функцию поиска пресечения множеств аналогов разных свойств для каждого артикула
        art_subs_about = intersection(**art_subs_props_about)
        # возвращаем словарь, в котором для каждого артикула записано множесво аналогов свойств "неполного соответствия"
        return art_subs_about
        
    # в противном случае для каждого артикула мы записываем множество всех артикулов, чтобы в дальнейшем
    # при пересечении это множество не могло негативно повлиять на кол-во полных аналогов артикула
    except:
        art_subs_about = {}
        for art in df['Код2'].unique():
            art_subs_about[art] = set(df['Код2'].unique())
        return art_subs_about

In [382]:
# Функция подбора аналогов по свойствам категории "полное соответсвие"

def subs_abs(df, group):
    # проверяем, есть ли правила "полного соответсвия" для этой группы
    try:
        rules_abs[group] # если есть группа в словаре правил "неполного соответсвия"
        art_subs_props_abs = {}

        for art in df['Код2'].unique(): # для каждого артикула
            prop_dict = {}
            
            for prop in rules_abs[group]: # для каждого свойства этого артикула
                value = df[(df['Код2'] == art) & (df['Свойство'] == prop)]['Значение'].tolist()[0] # мы записываем значение
                
                subs = set()
                for art_sub in df['Код2'].unique(): # каждый артикул рассматриваем в качестве аналога
                    if art == art_sub:
                        continue
                    else:
                        value_subs = df_art_list[(df_art_list['Код2'] == art_sub) &
                                                 (df_art_list['Свойство'] == prop)]['Значение'].tolist()[0]
                    if value == value_subs: # значения артикула и аналога равны, то мы добавляем аналог в мно-во аналогов
                        subs.add(art_sub)
                
                prop_dict[prop] = subs # заполняем словарь свойств аналогами
            art_subs_props_abs[art] = prop_dict # заполняем словарь артикулов словарями свойств
        art_subs_abs = intersection(**art_subs_props_abs) # для каждого арт-ла находим прересечение мно-в аналогов по свойствам
        return art_subs_abs
        
    except:
        art_subs_abs = {}
        for art in df['Код2'].unique():
            art_subs_abs[art] = set(df['Код2'].unique())
        return art_subs_abs

In [383]:
# Функция пересечения множеств артикулов аналогов по правилам цена, "полное" и "неполное" соответствие

def intersection_paa(dict1, dict2, dict3):
    art_subs_paa = {}
    for art in dict1.keys():
        set1 = dict1[art]
        set2 = dict2[art]
        set3 = dict3[art]
        art_subs_paa[art] = set1.intersection(set2, set3)
    return art_subs_paa

In [384]:
# функция составления множеств артикулов, в наименование которых входит или не входит слово

def contains(df, group):
    words ={} # словарь, в который для каждого слова из правила мы добавляем множество артикулов, которые соответствуют этому слову

    try:
        rules_contains[group] # проверяем есть ли рассматриваемая группа в правилах проверки слов
        
        # рассматриваем каждое слово из правила и записываем множество артикулов
        for word in rules_contains[group]:
            art_contains = set(df[df['Номенклатура'].str.contains(word, case = False)]['Код2'].unique())
            words[word] = art_contains

        list_arts_contains = [] # список артикулов, для которых удалось найти вхождения слов
        for value in words.values():
            list_arts_contains.append(value)

        all_arts = set(df['Код2'].unique())
        
        # множество артикулов, для которых не удалось найти вхожднение слова
        words['not_contain'] = all_arts.difference(*list_arts_contains)
    
    except:
        words['not_contain'] = set(df['Код2'].unique())

    return words

In [385]:
# Функция, которая для каждого артикула ищет пресечения между аналогами по свойствам и аналогами по вхождению слов

def prop_name_intersection(prop_dict, word_dict):

    art_subs_end = {} # словарь, куда мы записываем итоговое пересечение артикулов аналогов
    for art in prop_dict.keys(): # для каждого артикула
        for word in word_dict.keys(): # для каждого слова
            if art in word_dict[word]: # если артикул входит в множество артикулов, пренадлежащих слову
                subs_set = prop_dict[art].intersection(word_dict[word]) # мы ищем пересечения множества аналогов по свойсту с множеством артикулов, входящих в слово
                art_subs_end[art] = str(subs_set)
    # возвращаем словарь, в котором для каждого артикула записано множество основных аналогов            
    return art_subs_end

### Тестирование или подбор аналогов для выбранной группы

In [364]:
group = 'Переставные клещи'

In [376]:
df_art_list = act_art(group)

None
Количество артикулов, которые пренадлежат родителю "Переставные клещи": 44
Количество артикулов, которые попали в выборку: 2665

Согласно установленным правилам в отбор попадает слишком большое артикулов,
которые не пренадлежат родителю "Переставные клещи". Кажется, что в этом случае правила слишком общие и их стоит уточнить.
Установим ограничение на выборку только из соответствующего родителя "Переставные клещи"

Переставные клещи
Количество артикулов, которые пренадлежат родителю "Переставные клещи": 44
Количество артикулов, которые попали в выборку: 33



In [386]:
art_subs_price = subs_price(df_art_list)

In [174]:
art_subs_about = subs_about(df_art_list, group)

In [175]:
art_subs_abs = subs_abs(df_art_list, group)

In [176]:
art_subs_paa = intersection_paa(art_subs_price, art_subs_about, art_subs_abs)

In [177]:
art_name = contains(df = df_art_list, group = group)

In [198]:
art_subs_end = prop_name_intersection(art_subs_paa, art_name)

In [220]:
df_test = pd.DataFrame(art_subs_end, index = ['test']).T['test'].str.split(',', expand = True).reset_index()

In [313]:
df_list = []
for i in df_test.index:
    analog = pd.DataFrame(df_test.loc[i])
    osn = pd.DataFrame({'osn':i}, index = ['0'])
    merged = analog.merge(osn, how = 'cross')
    merged.rename(columns = {i:'analog'}, inplace = True)
    df_list.append(merged)

df_concat = pd.concat(df_list).reset_index(drop = True)
df_concat['analog1'] = df_concat['analog'].str.strip("{}'' ")
df_concat[df_concat['analog1'] != 'set()'].dropna()[['osn', 'analog1']].head(1)

,osn,analog1
18,437,42094


### Подбор аналогов для всех групп

In [394]:
start_time = time.time()

df_list = []
for group in rules.keys():
    df_art_list = act_art(group)
    art_subs_price = subs_price(df_art_list)
    art_subs_about = subs_about(df_art_list, group)
    art_subs_abs = subs_abs(df_art_list, group)
    art_subs_paa = intersection_paa(art_subs_price, art_subs_about, art_subs_abs)
    art_name = contains(df = df_art_list, group = group)
    art_subs_end = prop_name_intersection(art_subs_paa, art_name)
    
    df_preload = pd.DataFrame(art_subs_end, index = ['test']).T['test'].str.split(',', expand = True)
    for i in df_preload.index:
        analog = pd.DataFrame(df_preload.loc[i])
        osnovnoy = pd.DataFrame({'osn':i}, index = ['0'])
        merged = analog.merge(osnovnoy, how = 'cross')
        merged.rename(columns = {i:'analog'}, inplace = True)
        df_list.append(merged)
        
df_concat = pd.concat(df_list).reset_index(drop = True)
df_concat['analog1'] = df_concat['analog'].str.strip("{}'' ")
df_load = df_concat[df_concat['analog1'] != 'set()'].dropna()[['osn', 'analog1']]
df_load.to_excel(path+'Основные аналоги.xlsx')

end_time = time.time()
print(f'Run time: {(end_time - start_time)/60}')

Количество артикулов, которые пренадлежат родителю "Трещотки": 142
Количество артикулов, которые попали в выборку: 105

Количество артикулов, которые пренадлежат родителю "Наборы ключей": 106
Количество артикулов, которые попали в выборку: 76

Количество артикулов, которые пренадлежат родителю "Наборы торцевых головок": 162
Количество артикулов, которые попали в выборку: 65

Количество артикулов, которые пренадлежат родителю "Наборы шестигранных ключей": 69
Количество артикулов, которые попали в выборку: 147

Количество артикулов, которые пренадлежат родителю "Наборы бит": 46
Количество артикулов, которые попали в выборку: 8

Количество артикулов, которые пренадлежат родителю "Обжим": 142
Количество артикулов, которые попали в выборку: 20

Количество артикулов, которые пренадлежат родителю "Воротки ": 130
Количество артикулов, которые попали в выборку: 102

Количество артикулов, которые пренадлежат родителю "Наборы отверток": 34
Количество артикулов, которые попали в выборку: 42

Колич

# Тесты

In [412]:
sorted(act_items[act_items['Свойство'].str.contains('шаг резьбы', case = False)]['Значение'].unique())

['0,25 мм',
 '0,3 мм',
 '0,35 мм',
 '0,4 мм',
 '0,45 мм',
 '0,5 мм',
 '0,6 мм',
 '0,7 мм',
 '0,75 мм',
 '0,8 мм',
 '0,8-1,75 мм',
 '0,8-3 мм',
 '0,9 мм',
 '1',
 '1 tpi',
 '1 мм',
 '1,25',
 '1,25 мм',
 '1,25-1,5 мм',
 '1,27',
 '1,5',
 '1,5 tpi',
 '1,5 мм',
 '1,75',
 '1,75 мм',
 '1-3,5 мм; 4-24 tpi',
 '1.5',
 '1.75',
 '10',
 '10 ниток на дюйм',
 '11',
 '11 ниток на дюйм',
 '11,5 ниток на дюйм',
 '12 ниток на дюйм',
 '13 tpi',
 '13 ниток на дюйм',
 '14',
 '14 ниток на дюйм',
 '16',
 '16 ниток на дюйм',
 '18 ниток на дюйм',
 '19',
 '19 ниток на дюйм',
 '2',
 '2 tpi',
 '2 мм',
 '2,5',
 '2,5 мм',
 '2.5',
 '20 ниток на дюйм',
 '22 ниток на дюйм',
 '24 ниток на дюйм',
 '26 ниток на дюйм',
 '27 ниток на дюйм',
 '28 ниток на дюйм',
 '3',
 '3 мм',
 '3,5',
 '3,5 мм',
 '3.1/2 ниток на дюйм',
 '32 ниток на дюйм',
 '36 ниток на дюйм',
 '4',
 '4 мм',
 '4 ниток на дюйм',
 '4,5 мм',
 '4,5 ниток на дюйм',
 '4.1/2 ниток на дюйм',
 '4.5',
 '40 ниток на дюйм',
 '44 ниток на дюйм',
 '48 ниток на дюйм',
 '5',

In [414]:
len(act_items['Свойство'].unique())

1394